In [ ]:
import os


In [61]:
template = """
You are an expert SQL assistant.

Based on the database schema below, return **only a valid, executable SQL query** that answers the user's question.
Do **not** explain anything.
Do **not** return anything else — only the SQL.

Schema:
{schema}

Question: {question}

SQL:
"""
prompt = ChatPromptTemplate.from_template(template.strip())


In [62]:
prompt.format(schema="myschema",question="how many users are there?")

"Human: You are an expert SQL assistant.\n\nBased on the database schema below, return **only a valid, executable SQL query** that answers the user's question.\nDo **not** explain anything.\nDo **not** return anything else — only the SQL.\n\nSchema:\nmyschema\n\nQuestion: how many users are there?\n\nSQL:"

In [ ]:
from langchain_community.utilities import SQLDatabase
# if you are using MySQL
mysql_uri = ""

db = SQLDatabase.from_uri(mysql_uri)

In [64]:
db.run("SELECT * FROM artist limit 5;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]"

In [65]:
def get_schema(_):
    return db.get_table_info()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="mistralai/mistral-small-3.2-24b-instruct",  # full model name from OpenRouter
    openai_api_key="",
    openai_api_base=""     # ✅ Correct base URL
) 

# Your LangChain SQL chain
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["```", "SQLResult:", "\n```"])
    | StrOutputParser()
)


In [67]:
user_question = 'how many albums are there in the database?'
sql_chain.invoke({"question": user_question})

# 'SELECT COUNT(*) AS TotalAlbums\nFROM Album;'


'SELECT COUNT(*) FROM album;'

In [68]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [69]:
def run_query(query):
    return db.run(query)


In [74]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | llm
    | StrOutputParser()
)


In [75]:
user_question = 'are there 347 albums in the data?'
full_chain.invoke({"question": user_question})

# 'There are 347 albums in the database.'


'Based on the SQL query and response, there are indeed 347 albums in the data. The query counted the total number of records in the `album` table, which returned a result of 347.'